In [26]:
%%writefile filter_app.py
#imports
import cv2
import numpy as np
import streamlit as st
from PIL import Image

#---- Bulid our filters ---------------------------------------------------------------------------
#RGB --> Gray
def black_white(img):
    gray_image = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return gray_image

#brightness
def brightness(img,level):
    bright = cv2.convertScaleAbs(img, beta = level)
    return bright

#style image
def style_image(img,sigma_s= 10 ,sigma_r= 0.1): # sigma_s --> colors | sigma_r --> details
    blur_image = cv2.GaussianBlur(img, (5,5), 0,0)
    style_img = cv2.stylization(blur_image,sigma_s =sigma_s , sigma_r=sigma_r)
    return style_img


def HDR(img,level,sigma_s= 10 ,sigma_r= 0.1):
    bright = cv2.convertScaleAbs(img, beta = level)
    hd_image = cv2.detailEnhance(bright,sigma_s =sigma_s , sigma_r=sigma_r)


def vintage(img,level = 2):
    height, width = img.shape[:2]
    x_kernal = cv2.getGaussianKernel(width,width/level)
    y_kernal = cv2.getGaussianKernel(height,height/level)
    kernal = x_kernal.T * y_kernal
    mask = kernal/kernal.max()
    image_copy = np.copy(img)
    for i in range(3):
        image_copy[:,:,i] = image_copy[:,:,i]* mask
    return image_copy
#--------------------------------------------------------------------------------------#
#GUI
st.title('Image Filters App')
upload = st.file_uploader('Choose an Image', type = ['png','jpg','jpeg'])

if upload is not None:
    #setup my img
    img= Image.open(upload)
    img = np.array(img)
    img= cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    original_img , output_img = st.columns(2)


    with original_img:
        st.header('Original Image')
        st.image(img, channels = 'BGR')
    
    st.header('Filters List')
    options = st.selectbox('Select Filter: ',('None','black_white','brightness','style_image','HDR','vintage'))
    
    output_flag = 1
    color = 'BGR'

    if options == 'None':
        output_flag = 0
        output = img

    elif options =='black_white':
        output = black_white(img)
        color = 'GRAY'
        
    elif options == 'brightness':
        level = st.slider('brightness', -50,50,10,step =5)
        output = brightness(img,level)

    elif options == 'style_image':
        sigma_s = st.slider('sigma_s', 0,200,100,step =10)
        sigma_r = st.slider('sigma_r', 0,1,2)
        output= style_image(img,sigma_s,sigma_r)

    elif options == 'HDR':
        level = st.slider('brightness', -50,50,10,step =5)
        output=  HDR(img,level)

    elif options == 'vintage':
        level = st.slider('level', 0,5,1,step =1)
        output = vintage(img,level)

    with output_img:
        st.header('output Image')
        if color == "BGR":
            output = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        st.image(output)

Overwriting filter_app.py
